In [18]:
import finnhub
import yfinance as yf
import pandas as pd
import json
import time
from datetime import datetime, timedelta

# Initialize the Finnhub client with your API key
finnhub_client = finnhub.Client(api_key='ct4hebhr01qo7vqamo40ct4hebhr01qo7vqamo4g')  # Replace with your actual API key

def flatten_financials_section(section_data, prefix, desired_keys):
    """
    Flattens the financial section (bs, cf, ic) data, prefixing keys.
    Only includes specified desired keys.
    """
    flat_data = {}
    if isinstance(section_data, dict):
        for key, value in section_data.items():
            if key in desired_keys:
                flat_data[f"{prefix}_{key}"] = value
    elif isinstance(section_data, list):
        for item in section_data:
            for key, value in item.items():
                if key in desired_keys:
                    flat_data[f"{prefix}_{key}"] = value
    return flat_data

def get_finnhub_data(symbol):
    data = {}
    
    # Fetch Company Profile
    try:
        profile = finnhub_client.company_profile2(symbol=symbol)
        if profile:
            data['Country'] = profile.get('country', 'N/A')
            data['Currency'] = profile.get('currency', 'N/A')
            data['Industry_finnhub'] = profile.get('finnhubIndustry', 'N/A')
            data['IPO_Date'] = profile.get('ipo', 'N/A')
            data['Logo_URL'] = profile.get('logo', 'N/A')
            data['Market_Cap_finnhub'] = profile.get('marketCapitalization', 'N/A')
            data['Name_finnhub'] = profile.get('name', 'N/A')
            data['Phone'] = profile.get('phone', 'N/A')
            data['Outstanding_Shares'] = profile.get('shareOutstanding', 'N/A')
            data['Ticker'] = symbol  # Ensure Ticker is included
            data['Website'] = profile.get('weburl', 'N/A')
        else:
            print(f"No Company Profile data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching company profile for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch Stock Quote
    try:
        quote = finnhub_client.quote(symbol)
        if quote:
            data['Current_Price'] = quote.get('c', 'N/A')
            data['High'] = quote.get('h', 'N/A')
            data['Low'] = quote.get('l', 'N/A')
            data['Open'] = quote.get('o', 'N/A')
            data['Previous_Close'] = quote.get('pc', 'N/A')
            data['Timestamp'] = pd.to_datetime(quote.get('t', 0), unit='s').strftime('%Y-%m-%d %H:%M:%S') if quote.get('t') else 'N/A'
        else:
            print(f"No Stock Quote data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching stock quote for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch Financials Reported
    try:
        financials = finnhub_client.financials_reported(symbol=symbol, freq='annual')
        if financials and 'data' in financials and len(financials['data']) > 0:
            latest_financial = financials['data'][0]  # Get the most recent report
            report = latest_financial.get('report', {})
            
            # Desired keys for each financial section
            desired_bs_keys = ['Assets', 'Liabilities', 'InventoryNet']
            desired_cf_keys = ['NetIncomeLoss', 'InterestPaidNet']
            desired_ic_keys = ['GrossProfit', 'NetIncomeLoss', 'OperatingExpenses']
            
            # Flatten 'ic', 'bs', 'cf' sections
            for section_key, desired_keys, prefix in [
                ('ic', desired_ic_keys, 'Income'),
                ('bs', desired_bs_keys, 'Balance'),
                ('cf', desired_cf_keys, 'CashFlow')
            ]:
                section_data = report.get(section_key, {})
                flat_section_data = flatten_financials_section(section_data, prefix=prefix, desired_keys=desired_keys)
                data.update(flat_section_data)
        else:
            print(f"No Financials Reported data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching financials reported for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch Recommendation Trends
    try:
        rec_trends = finnhub_client.recommendation_trends(symbol=symbol)
        if rec_trends and len(rec_trends) > 0:
            latest_trend = rec_trends[0]  # Get the most recent trend
            data['Buy'] = latest_trend.get('buy', 'N/A')
            data['Hold'] = latest_trend.get('hold', 'N/A')
            data['Sell'] = latest_trend.get('sell', 'N/A')
            data['Strong_Buy'] = latest_trend.get('strongBuy', 'N/A')
            data['Strong_Sell'] = latest_trend.get('strongSell', 'N/A')
            data['Recommendation_Period'] = latest_trend.get('period', 'N/A')
        else:
            print(f"No Recommendation Trends data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching recommendation trends for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch General News
    try:
        general_news = finnhub_client.general_news(category='general', min_id=0)
        if general_news and len(general_news) > 0:
            # Aggregate news into single cell
            news_combined = '; '.join([f"{item.get('headline', 'N/A')}: {item.get('summary', 'N/A')}" for item in general_news])
            data['General_News'] = news_combined
        else:
            data['General_News'] = 'N/A'
            print(f"No General News found.")
    except Exception as e:
        data['General_News'] = 'N/A'
        print(f"Error fetching general news: {e}")
    time.sleep(1)
    
    # Fetch Company News
    try:
        end_date = datetime(2023, 12, 31)
        start_date = (end_date - timedelta(days=365)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        company_news = finnhub_client.company_news(symbol, _from=start_date, to=end_date)
        if company_news and len(company_news) > 0:
            news_combined = '; '.join([f"{item.get('headline', 'N/A')}: {item.get('summary', 'N/A')}" for item in company_news])
            data['Company_News'] = news_combined
        else:
            data['Company_News'] = 'N/A'
            print(f"No Company News found for {symbol}.")
    except Exception as e:
        data['Company_News'] = 'N/A'
        print(f"Error fetching company news for {symbol}: {e}")
    time.sleep(1)
    
    return data

def yfinance_data(ticker):
    """
    Fetches data from Yahoo Finance for the given ticker symbol.
    """
    data = {}
    try:
        stock = yf.Ticker(ticker)
        
        # Fetch basic information
        info = stock.info
        data['Company_Name_yf'] = info.get('longName', 'N/A')
        data['Sector_yf'] = info.get('sector', 'N/A')
        data['Industry_yf'] = info.get('industry', 'N/A')
        data['Market_Cap_yf'] = info.get('marketCap', 'N/A')
        data['52_Week_High'] = info.get('fiftyTwoWeekHigh', 'N/A')
        data['52_Week_Low'] = info.get('fiftyTwoWeekLow', 'N/A')
        data['Dividend_Yield'] = info.get('dividendYield', 'N/A')
        
        # Fetch historical data from 2013-01-01 to current date monthly
        start_date = '2013-01-01'
        end_date = datetime.today().strftime('%Y-%m-%d')
        historical_data = stock.history(start=start_date, end=end_date, interval='1mo')
        
        if not historical_data.empty:
            # Reset index to get 'Date' as a column
            historical_data.reset_index(inplace=True)
            # Keep only 'Date' and 'Close'
            historical_close = historical_data[['Date', 'Close']].dropna()
            # Convert to dictionary
            historical_close_dict = historical_close.set_index('Date')['Close'].to_dict()
            # Convert dates to string format
            historical_close_dict = {date.strftime('%Y-%m-%d'): price for date, price in historical_close_dict.items()}
            data['Historical_Close_Prices'] = historical_close_dict
        else:
            data['Historical_Close_Prices'] = {}
            print(f"No historical data found for {ticker}.")
        
    except Exception as e:
        print(f"An error occurred with yfinance for {ticker}: {e}")
        data = {}
    
    return data

def save_to_csv(company_data, historical_prices, main_filename='stock_data.csv', prices_filename='historical_prices.csv'):
    """
    Saves company data and historical prices to their respective CSV files.
    """
    # Create a DataFrame for company data
    company_df = pd.DataFrame([company_data])

    # Replace NaN or missing values with 'N/A'
    company_df.fillna('N/A', inplace=True)

    # Define desired columns
    desired_columns = [
        'Ticker', 'Company_Name_yf', 'Sector_yf', 'Industry_yf', 'Market_Cap_yf',
        '52_Week_High', '52_Week_Low', 'Dividend_Yield',
        'Balance_Assets', 'Balance_Liabilities', 'CashFlow_NetIncomeLoss', 'Income_GrossProfit',
        'Current_Price', 'High', 'Low', 'Open', 'Previous_Close',
        'Buy', 'Hold', 'Sell', 'Strong_Buy', 'Strong_Sell', 'Recommendation_Period',
        'General_News', 'Company_News'
    ]

    # Add any missing columns with 'N/A'
    for col in desired_columns:
        if col not in company_df.columns:
            company_df[col] = 'N/A'

    # Reorder columns
    company_df = company_df[desired_columns]
    
    # Save or append to the main CSV
    try:
        existing_company_df = pd.read_csv(main_filename)
        # Append new data
        company_df = pd.concat([existing_company_df, company_df], ignore_index=True)
        # Drop duplicates based on 'Ticker'
        company_df.drop_duplicates(subset='Ticker', keep='last', inplace=True)
    except FileNotFoundError:
        pass  # File doesn't exist, will create a new one
    
    # Save the main company data
    company_df.to_csv(main_filename, index=False)
    print(f"Company data saved to {main_filename}")
    
    # Prepare historical prices DataFrame
    if historical_prices:
        historical_prices_list = []
        for date, price in historical_prices.items():
            historical_prices_list.append({
                'Date': date,
                'Company_Name': company_data.get('Company_Name_yf', 'N/A'),
                'Close_Price': price
            })
        historical_prices_df = pd.DataFrame(historical_prices_list)
        
        # Replace NaN or missing values with 'N/A'
        historical_prices_df.fillna('N/A', inplace=True)
        
        # Save or append to the historical prices CSV
        try:
            existing_prices_df = pd.read_csv(prices_filename)
            historical_prices_df = pd.concat([existing_prices_df, historical_prices_df], ignore_index=True)
            # Drop duplicates based on 'Date' and 'Company_Name'
            historical_prices_df.drop_duplicates(subset=['Date', 'Company_Name'], keep='last', inplace=True)
        except FileNotFoundError:
            pass  # File doesn't exist, will create a new one
        
        # Save the historical prices
        historical_prices_df.to_csv(prices_filename, index=False)
        print(f"Historical prices saved to {prices_filename}")
    else:
        print("No historical prices to save.")

if __name__ == '__main__':
    # List of symbols to process
    symbols = ['HLAL', 'BOUBYAN.KW','BRIS.JK','1120.SR','QIBK.QA','PNC','USB','CME','ICE','MCO']  # Add more symbols as needed
    
    for symbol in symbols:
        print(f"\nProcessing data for {symbol}...")
        
        # Get data from Finnhub
        finnhub_data = get_finnhub_data(symbol)
        
        # Get data from Yahoo Finance
        yfinance_result = yfinance_data(symbol)
        
        # Merge Finnhub and Yahoo Finance data
        combined_data = {**finnhub_data, **yfinance_result}
        
        # Save to CSV
        save_to_csv(combined_data, combined_data.get('Historical_Close_Prices', {}))





Processing data for HLAL...
No Company Profile data found for HLAL.
No Financials Reported data found for HLAL.
No Recommendation Trends data found for HLAL.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for BOUBYAN.KW...
No Financials Reported data found for BOUBYAN.KW.
No Company News found for BOUBYAN.KW.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for BRIS.JK...
Error fetching company profile for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Error fetching stock quote for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Error fetching financials reported for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Error fetching recommendation trends for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.


KeyboardInterrupt: 

In [23]:
import finnhub
import yfinance as yf
import pandas as pd
import json
import time
from datetime import datetime, timedelta

# Initialize the Finnhub client with your API key
finnhub_client = finnhub.Client(api_key='ct4hebhr01qo7vqamo40ct4hebhr01qo7vqamo4g')  # Replace with your actual API key

def flatten_financials_section(section_data, prefix, desired_keys):
    """
    Flattens the financial section (bs, cf, ic) data, prefixing keys.
    Only includes specified desired keys.
    """
    flat_data = {}
    if isinstance(section_data, dict):
        for key, value in section_data.items():
            if key in desired_keys:
                flat_data[f"{prefix}_{key}"] = value
    elif isinstance(section_data, list):
        for item in section_data:
            for key, value in item.items():
                if key in desired_keys:
                    flat_data[f"{prefix}_{key}"] = value
    return flat_data

def get_finnhub_data(symbol):
    data = {}
    data['Ticker'] = symbol  # Ensure Ticker is included
    
    # Fetch Company Profile
    try:
        profile = finnhub_client.company_profile2(symbol=symbol)
        if profile:
            data['Country'] = profile.get('country', 'N/A')
            data['Currency'] = profile.get('currency', 'N/A')
            data['Industry_finnhub'] = profile.get('finnhubIndustry', 'N/A')
            data['IPO_Date'] = profile.get('ipo', 'N/A')
            data['Logo_URL'] = profile.get('logo', 'N/A')
            data['Market_Cap_finnhub'] = profile.get('marketCapitalization', 'N/A')
            data['Name_finnhub'] = profile.get('name', 'N/A')
            data['Phone'] = profile.get('phone', 'N/A')
            data['Outstanding_Shares'] = profile.get('shareOutstanding', 'N/A')
            data['Website'] = profile.get('weburl', 'N/A')
        else:
            print(f"No Company Profile data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching company profile for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch Stock Quote
    try:
        quote = finnhub_client.quote(symbol)
        if quote:
            data['Current_Price'] = quote.get('c', 'N/A')
            data['High'] = quote.get('h', 'N/A')
            data['Low'] = quote.get('l', 'N/A')
            data['Open'] = quote.get('o', 'N/A')
            data['Previous_Close'] = quote.get('pc', 'N/A')
            data['Timestamp'] = pd.to_datetime(quote.get('t', 0), unit='s').strftime('%Y-%m-%d %H:%M:%S') if quote.get('t') else 'N/A'
        else:
            print(f"No Stock Quote data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching stock quote for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch Financials Reported
    try:
        financials = finnhub_client.financials_reported(symbol=symbol, freq='annual')
        if financials and 'data' in financials and len(financials['data']) > 0:
            latest_financial = financials['data'][0]  # Get the most recent report
            report = latest_financial.get('report', {})
            
            # Desired keys for each financial section
            desired_bs_keys = ['Assets', 'Liabilities', 'InventoryNet']
            desired_cf_keys = ['NetIncomeLoss', 'InterestPaidNet']
            desired_ic_keys = ['GrossProfit', 'NetIncomeLoss', 'OperatingExpenses']
            
            # Flatten 'ic', 'bs', 'cf' sections
            for section_key, desired_keys, prefix in [
                ('ic', desired_ic_keys, 'Income'),
                ('bs', desired_bs_keys, 'Balance'),
                ('cf', desired_cf_keys, 'CashFlow')
            ]:
                section_data = report.get(section_key, {})
                flat_section_data = flatten_financials_section(section_data, prefix=prefix, desired_keys=desired_keys)
                data.update(flat_section_data)
        else:
            print(f"No Financials Reported data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching financials reported for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch Recommendation Trends
    try:
        rec_trends = finnhub_client.recommendation_trends(symbol=symbol)
        if rec_trends and len(rec_trends) > 0:
            latest_trend = rec_trends[0]  # Get the most recent trend
            data['Buy'] = latest_trend.get('buy', 'N/A')
            data['Hold'] = latest_trend.get('hold', 'N/A')
            data['Sell'] = latest_trend.get('sell', 'N/A')
            data['Strong_Buy'] = latest_trend.get('strongBuy', 'N/A')
            data['Strong_Sell'] = latest_trend.get('strongSell', 'N/A')
            data['Recommendation_Period'] = latest_trend.get('period', 'N/A')
        else:
            print(f"No Recommendation Trends data found for {symbol}.")
    except Exception as e:
        print(f"Error fetching recommendation trends for {symbol}: {e}")
    time.sleep(1)
    
    # Fetch General News
    try:
        general_news = finnhub_client.general_news(category='general', min_id=0)
        if general_news and len(general_news) > 0:
            # Aggregate news into single cell
            news_combined = '; '.join([f"{item.get('headline', 'N/A')}: {item.get('summary', 'N/A')}" for item in general_news])
            data['General_News'] = news_combined
        else:
            data['General_News'] = 'N/A'
            print(f"No General News found.")
    except Exception as e:
        data['General_News'] = 'N/A'
        print(f"Error fetching general news: {e}")
    time.sleep(1)
    
    # Fetch Company News
    try:
        end_date = datetime(2023, 12, 31)
        start_date = (end_date - timedelta(days=365)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        company_news = finnhub_client.company_news(symbol, _from=start_date, to=end_date)
        if company_news and len(company_news) > 0:
            news_combined = '; '.join([f"{item.get('headline', 'N/A')}: {item.get('summary', 'N/A')}" for item in company_news])
            data['Company_News'] = news_combined
        else:
            data['Company_News'] = 'N/A'
            print(f"No Company News found for {symbol}.")
    except Exception as e:
        data['Company_News'] = 'N/A'
        print(f"Error fetching company news for {symbol}: {e}")
    time.sleep(1)
    
    return data

def yfinance_data(ticker):
    """
    Fetches detailed data from Yahoo Finance for the given ticker symbol.
    """
    data = {}
    data['Ticker'] = ticker  # Ensure Ticker is included

    try:
        stock = yf.Ticker(ticker)
        
        # Fetch basic stock information
        info = stock.info
        data['Company_Name_yf'] = info.get('longName', 'N/A')
        data['Sector_yf'] = info.get('sector', 'N/A')
        data['Industry_yf'] = info.get('industry', 'N/A')
        data['Market_Cap_yf'] = info.get('marketCap', 'N/A')
        
        # Valuation Metrics
        data['Trailing_PE'] = info.get('trailingPE', 'N/A')
        data['Forward_PE'] = info.get('forwardPE', 'N/A')
        data['Price_to_Book'] = info.get('priceToBook', 'N/A')
        
        # Profitability Metrics
        data['Profit_Margins'] = info.get('profitMargins', 'N/A')
        data['Gross_Margins'] = info.get('grossMargins', 'N/A')
        data['Operating_Margins'] = info.get('operatingMargins', 'N/A')
        
        # Efficiency Ratios
        data['Return_on_Assets'] = info.get('returnOnAssets', 'N/A')
        data['Return_on_Equity'] = info.get('returnOnEquity', 'N/A')
        
        # Risk Metrics
        data['Beta'] = info.get('beta', 'N/A')
        data['Enterprise_to_Ebitda'] = info.get('enterpriseToEbitda', 'N/A')
        
        # Income Statement Data
        data['Total_Revenue'] = info.get('totalRevenue', 'N/A')
        data['Revenue_Growth'] = info.get('revenueGrowth', 'N/A')
        data['Earnings_Growth'] = info.get('earningsGrowth', 'N/A')
        
        # Balance Sheet Data
        data['Total_Debt'] = info.get('totalDebt', 'N/A')
        data['Debt_to_Equity'] = info.get('debtToEquity', 'N/A')
        
        # Cash Flow Data
        data['Operating_Cashflow'] = info.get('operatingCashflow', 'N/A')
        data['Free_Cashflow'] = info.get('freeCashflow', 'N/A')
        
        # Dividend Information
        data['Dividend_Yield'] = info.get('dividendYield', 'N/A')
        data['Payout_Ratio'] = info.get('payoutRatio', 'N/A')
        
        # Other Metrics
        data['Enterprise_Value'] = info.get('enterpriseValue', 'N/A')
        data['Peg_Ratio'] = info.get('pegRatio', 'N/A')
        data['Book_Value_Per_Share'] = info.get('bookValue', 'N/A')
        data['Forward_EPS'] = info.get('forwardEps', 'N/A')
        
        # Historical data from 2013-01-01 to 2023-12-31
        start_date = '2013-01-01'
        end_date = '2023-12-31'
        historical_data = stock.history(start=start_date, end=end_date, interval='1mo')
        
        if not historical_data.empty:
            # Reset index to get 'Date' as a column
            historical_data.reset_index(inplace=True)
            # Keep only 'Date' and 'Close'
            historical_close = historical_data[['Date', 'Close']].dropna()
            # Convert to dictionary
            historical_close_dict = historical_close.set_index('Date')['Close'].to_dict()
            # Convert dates to string format
            historical_close_dict = {date.strftime('%Y-%m-%d'): price for date, price in historical_close_dict.items()}
            data['Historical_Close_Prices'] = historical_close_dict
        else:
            data['Historical_Close_Prices'] = {}
            print(f"No historical data found for {ticker}.")
        
    except Exception as e:
        print(f"An error occurred with yfinance for {ticker}: {e}")
        # Do not reset data; keep 'Ticker' key
        
    return data

def save_to_csv(company_data, historical_prices, main_filename='stock_data.csv', prices_filename='historical_prices.csv'):
    """
    Saves company data and historical prices to their respective CSV files.
    """
    # Create a DataFrame for company data
    company_df = pd.DataFrame([company_data])

    # Replace NaN or missing values with 'N/A'
    company_df.fillna('N/A', inplace=True)

    # Define desired columns
    desired_columns = [
        'Ticker', 'Company_Name_yf', 'Sector_yf', 'Industry_yf', 'Market_Cap_yf',
        'Trailing_PE', 'Forward_PE', 'Price_to_Book', 'Profit_Margins', 'Gross_Margins', 'Operating_Margins',
        'Return_on_Assets', 'Return_on_Equity', 'Beta', 'Enterprise_to_Ebitda',
        'Total_Revenue', 'Revenue_Growth', 'Earnings_Growth', 'Total_Debt',
        'Debt_to_Equity', 'Operating_Cashflow', 'Free_Cashflow', 'Dividend_Yield', 'Payout_Ratio',
        'Enterprise_Value', 'Peg_Ratio', 'Book_Value_Per_Share', 'Forward_EPS',
        'General_News', 'Company_News'
    ]

    # Add any missing columns with 'N/A'
    for col in desired_columns:
        if col not in company_df.columns:
            company_df[col] = 'N/A'

    # Reorder columns
    company_df = company_df[desired_columns]

    # Save or append to the main CSV
    try:
        existing_company_df = pd.read_csv(main_filename)
        # Append new data
        company_df = pd.concat([existing_company_df, company_df], ignore_index=True)
        # Drop duplicates based on 'Ticker'
        company_df.drop_duplicates(subset='Ticker', keep='last', inplace=True)
    except FileNotFoundError:
        pass  # File doesn't exist, will create a new one

    # Save the main company data
    company_df.to_csv(main_filename, index=False)
    print(f"Company data saved to {main_filename}")

    # Prepare historical prices DataFrame
    if historical_prices:
        historical_prices_list = []
        for date, price in historical_prices.items():
            historical_prices_list.append({
                'Date': date,
                'Company_Name': company_data.get('Company_Name_yf', 'N/A'),
                'Close_Price': price
            })
        historical_prices_df = pd.DataFrame(historical_prices_list)

        # Replace NaN or missing values with 'N/A'
        historical_prices_df.fillna('N/A', inplace=True)

        # Save or append to the historical prices CSV
        try:
            existing_prices_df = pd.read_csv(prices_filename)
            historical_prices_df = pd.concat([existing_prices_df, historical_prices_df], ignore_index=True)
            # Drop duplicates based on 'Date' and 'Company_Name'
            historical_prices_df.drop_duplicates(subset=['Date', 'Company_Name'], keep='last', inplace=True)
        except FileNotFoundError:
            pass  # File doesn't exist, will create a new one

        # Save the historical prices
        historical_prices_df.to_csv(prices_filename, index=False)
        print(f"Historical prices saved to {prices_filename}")
    else:
        print("No historical prices to save.")

if __name__ == '__main__':
    # List of symbols to process
    symbols = ['HLAL', 'BOUBYAN.KW', 'BRIS.JK', '1120.SR', 'QIBK.QA', 'PNC', 'USB', 'CME', 'ICE', 'MCO']  # Add more symbols as needed

    for symbol in symbols:
        print(f"\nProcessing data for {symbol}...")
        
        # Get data from Finnhub
        finnhub_data = get_finnhub_data(symbol)
        
        # Get data from Yahoo Finance
        yfinance_result = yfinance_data(symbol)
        
        # Merge Finnhub and Yahoo Finance data
        combined_data = {**finnhub_data, **yfinance_result}
        
        # Save to CSV
        save_to_csv(combined_data, combined_data.get('Historical_Close_Prices', {}))



Processing data for HLAL...
No Company Profile data found for HLAL.
No Financials Reported data found for HLAL.
No Recommendation Trends data found for HLAL.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for BOUBYAN.KW...
No Financials Reported data found for BOUBYAN.KW.
No Company News found for BOUBYAN.KW.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for BRIS.JK...
Error fetching company profile for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Error fetching stock quote for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Error fetching financials reported for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Error fetching recommendation trends for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Error fetching c

In [6]:
import finnhub
import yfinance as yf
import pandas as pd
import time
from datetime import datetime, timedelta

# Initialize the Finnhub client with your API key
finnhub_client = finnhub.Client(api_key='ct4hebhr01qo7vqamo40ct4hebhr01qo7vqamo4g')  # Replace with your actual API key

def get_finnhub_data(symbol):
    data = {}
    data['Ticker'] = symbol  # Ensure Ticker is included
    try:
        # Define the fixed end date as 2023-12-31
        end_date = datetime(2023, 12, 31)
        # Calculate the start date as one year before the end date
        start_date = (end_date - timedelta(days=365)).strftime('%Y-%m-%d')
        # Format the end date
        end_date = end_date.strftime('%Y-%m-%d')

        # Fetch company news using the specified date range
        company_news = finnhub_client.company_news(symbol, _from=start_date, to=end_date)
        if company_news and len(company_news) > 0:
            # Aggregate news into a single cell
            news_combined = '; '.join([f"{item.get('headline', 'N/A')}: {item.get('summary', 'N/A')}" for item in company_news])
            data['Company_News'] = news_combined
        else:
            data['Company_News'] = 'N/A'
            print(f"No Company News found for {symbol}.")
    except Exception as e:
        data['Company_News'] = 'N/A'
        print(f"Error fetching company news for {symbol}: {e}")
    return data

def yfinance_data(ticker):
    """
    Fetches detailed data from Yahoo Finance for the given ticker symbol.
    """
    data = {}
    data['Ticker'] = ticker  # Ensure Ticker is included

    try:
        stock = yf.Ticker(ticker)
        
        # Fetch basic stock information
        info = stock.info
        data['Company_Name_yf'] = info.get('longName', 'N/A')
        data['Sector_yf'] = info.get('sector', 'N/A')
        data['Industry_yf'] = info.get('industry', 'N/A')
        data['Market_Cap_yf'] = info.get('marketCap', 'N/A')
        
        # Valuation Metrics
        data['Trailing_PE'] = info.get('trailingPE', 'N/A')
        data['Forward_PE'] = info.get('forwardPE', 'N/A')
        data['Price_to_Book'] = info.get('priceToBook', 'N/A')
        
        # Profitability Metrics
        data['Profit_Margins'] = info.get('profitMargins', 'N/A')
        data['Gross_Margins'] = info.get('grossMargins', 'N/A')
        data['Operating_Margins'] = info.get('operatingMargins', 'N/A')
        
        # Efficiency Ratios
        data['Return_on_Assets'] = info.get('returnOnAssets', 'N/A')
        data['Return_on_Equity'] = info.get('returnOnEquity', 'N/A')
        
        # Risk Metrics
        data['Beta'] = info.get('beta', 'N/A')
        data['Enterprise_to_Ebitda'] = info.get('enterpriseToEbitda', 'N/A')
        
        # Income Statement Data
        data['Total_Revenue'] = info.get('totalRevenue', 'N/A')
        data['Revenue_Growth'] = info.get('revenueGrowth', 'N/A')
        data['Earnings_Growth'] = info.get('earningsGrowth', 'N/A')
        
        # Balance Sheet Data
        data['Total_Debt'] = info.get('totalDebt', 'N/A')
        data['Debt_to_Equity'] = info.get('debtToEquity', 'N/A')
        
        # Cash Flow Data
        data['Operating_Cashflow'] = info.get('operatingCashflow', 'N/A')
        data['Free_Cashflow'] = info.get('freeCashflow', 'N/A')
        
        # Dividend Information
        data['Dividend_Yield'] = info.get('dividendYield', 'N/A')
        data['Payout_Ratio'] = info.get('payoutRatio', 'N/A')
        
        # Other Metrics
        data['Enterprise_Value'] = info.get('enterpriseValue', 'N/A')
        data['Peg_Ratio'] = info.get('pegRatio', 'N/A')
        data['Book_Value_Per_Share'] = info.get('bookValue', 'N/A')
        data['Forward_EPS'] = info.get('forwardEps', 'N/A')
        
        # Historical data from 2013-01-01 to 2023-12-31
        start_date = '2013-01-01'
        end_date = '2023-12-31'
        historical_data = stock.history(start=start_date, end=end_date, interval='1mo')
        
        if not historical_data.empty:
            # Reset index to get 'Date' as a column
            historical_data.reset_index(inplace=True)
            # Keep only 'Date' and 'Close'
            historical_close = historical_data[['Date', 'Close']].dropna()
            # Convert to dictionary
            historical_close_dict = historical_close.set_index('Date')['Close'].to_dict()
            # Convert dates to string format
            historical_close_dict = {date.strftime('%Y-%m-%d'): price for date, price in historical_close_dict.items()}
            data['Historical_Close_Prices'] = historical_close_dict
        else:
            data['Historical_Close_Prices'] = {}
            print(f"No historical data found for {ticker}.")
        
    except Exception as e:
        print(f"An error occurred with yfinance for {ticker}: {e}")
        # Do not reset data; keep 'Ticker' key
        
    return data

def save_to_csv(company_data, historical_prices, main_filename='stock_data.csv', prices_filename='historical_prices.csv'):
    """
    Saves company data and historical prices to their respective CSV files.
    """
    # Create a DataFrame for company data
    company_df = pd.DataFrame([company_data])

    # Replace NaN or missing values with 'N/A'
    company_df.fillna('N/A', inplace=True)

    # Define desired columns
    desired_columns = [
        'Ticker', 'Company_Name_yf', 'Sector_yf', 'Industry_yf', 'Market_Cap_yf',
        'Trailing_PE', 'Forward_PE', 'Price_to_Book', 'Profit_Margins', 'Gross_Margins', 'Operating_Margins',
        'Return_on_Assets', 'Return_on_Equity', 'Beta', 'Enterprise_to_Ebitda',
        'Total_Revenue', 'Revenue_Growth', 'Earnings_Growth', 'Total_Debt',
        'Debt_to_Equity', 'Operating_Cashflow', 'Free_Cashflow', 'Dividend_Yield', 'Payout_Ratio',
        'Enterprise_Value', 'Peg_Ratio', 'Book_Value_Per_Share', 'Forward_EPS',
        'Company_News'  # Include Company_News from finnhub data
    ]

    # Add any missing columns with 'N/A'
    for col in desired_columns:
        if col not in company_df.columns:
            company_df[col] = 'N/A'

    # Reorder columns
    company_df = company_df[desired_columns]

    # Save or append to the main CSV
    try:
        existing_company_df = pd.read_csv(main_filename)
        # Append new data
        company_df = pd.concat([existing_company_df, company_df], ignore_index=True)
        # Drop duplicates based on 'Ticker'
        company_df.drop_duplicates(subset='Ticker', keep='last', inplace=True)
    except FileNotFoundError:
        pass  # File doesn't exist, will create a new one

    # Save the main company data
    company_df.to_csv(main_filename, index=False)
    print(f"Company data saved to {main_filename}")

    # Prepare historical prices DataFrame
    if historical_prices:
        historical_prices_list = []
        for date, price in historical_prices.items():
            historical_prices_list.append({
                'Date': date,
                'Company_Name': company_data.get('Company_Name_yf', 'N/A'),
                'Close_Price': price
            })
        historical_prices_df = pd.DataFrame(historical_prices_list)

        # Replace NaN or missing values with 'N/A'
        historical_prices_df.fillna('N/A', inplace=True)

        # Save or append to the historical prices CSV
        try:
            existing_prices_df = pd.read_csv(prices_filename)
            historical_prices_df = pd.concat([existing_prices_df, historical_prices_df], ignore_index=True)
            # Drop duplicates based on 'Date' and 'Company_Name'
            historical_prices_df.drop_duplicates(subset=['Date', 'Company_Name'], keep='last', inplace=True)
        except FileNotFoundError:
            pass  # File doesn't exist, will create a new one

        # Save the historical prices
        historical_prices_df.to_csv(prices_filename, index=False)
        print(f"Historical prices saved to {prices_filename}")
    else:
        print("No historical prices to save.")

if __name__ == '__main__':
    # List of symbols to process
    symbols = ['HLAL', 'BOUBYAN.KW', 'BRIS.JK', '1120.SR', 'QIBK.QA', 'PNC', 'USB', 'CME', 'ICE', 'MCO', 'MJ']  # Add more symbols as needed

    for symbol in symbols:
        print(f"\nProcessing data for {symbol}...")
        
        # Get data from Finnhub
        finnhub_data = get_finnhub_data(symbol)
        
        # Get data from Yahoo Finance
        yfinance_result = yfinance_data(symbol)
        
        # Merge data from both sources
        combined_data = {**yfinance_result, **finnhub_data}
        
        # Save to CSV
        save_to_csv(combined_data, combined_data.get('Historical_Close_Prices', {}))



Processing data for HLAL...
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for BOUBYAN.KW...
No Company News found for BOUBYAN.KW.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for BRIS.JK...
Error fetching company news for BRIS.JK: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for 1120.SR...
Error fetching company news for 1120.SR: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing data for QIBK.QA...
Error fetching company news for QIBK.QA: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Company data saved to stock_data.csv
Historical prices saved to historical_prices.csv

Processing 

In [26]:
import finnhub
import yfinance as yf
import time

# Initialize the Finnhub client with your API key
finnhub_client = finnhub.Client(api_key='ct4hebhr01qo7vqamo40ct4hebhr01qo7vqamo4g')  # Replace with your actual API key

def get_finnhub_data(symbol):
    data = {}
    
    # Fetch Company Profile
    try:
        profile = finnhub_client.company_profile2(symbol=symbol)
        data['Company Profile'] = profile
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Company Profile'] = f"Error fetching company profile: {e}"
    time.sleep(1)
    
    # Fetch Financials Reported
    try:
        financials_reported = finnhub_client.financials_reported(symbol=symbol, freq='annual')
        data['Financials Reported'] = financials_reported
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Financials Reported'] = f"Error fetching financials reported: {e}"
    time.sleep(1)
    
    # Fetch Filings
    try:
        filings = finnhub_client.filings(symbol=symbol, _from="2013-01-01", to="2023-12-31")
        data['Filings'] = filings
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Filings'] = f"Error fetching filings: {e}"
    time.sleep(1)
    
    # Fetch Company Peers
    try:
        peers = finnhub_client.company_peers(symbol)
        data['Company Peers'] = peers
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Company Peers'] = f"Error fetching company peers: {e}"
    time.sleep(1)
    
    # Fetch Basic Financials
    try:
        basic_financials = finnhub_client.company_basic_financials(symbol, 'all')
        data['Basic Financials'] = basic_financials
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Basic Financials'] = f"Error fetching basic financials: {e}"
    time.sleep(1)
    
    # Fetch Company News
    try:
        company_news = finnhub_client.company_news(symbol, _from="2022-12-31", to="2023-12-31")
        data['Company News'] = company_news
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Company News'] = f"Error fetching company news: {e}"
    time.sleep(1)
    
    # Fetch Recommendation Trends
    try:
        recommendation_trends = finnhub_client.recommendation_trends(symbol)
        data['Recommendation Trends'] = recommendation_trends
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Recommendation Trends'] = f"Error fetching recommendation trends: {e}"
    time.sleep(1)
    
    # Fetch Quote
    try:
        quote = finnhub_client.quote(symbol)
        data['Quote'] = quote
    except finnhub.exceptions.FinnhubAPIException as e:
        data['Quote'] = f"Error fetching quote: {e}"
    time.sleep(1)
    
    return data

def get_yfinance_data(ticker):
    data = {}
    try:
        # Download the stock data
        stock = yf.Ticker(ticker)
        
        # Fetch basic stock information
        info = stock.info
        data['Company Name'] = info.get('longName', 'N/A')
        data['Sector'] = info.get('sector', 'N/A')
        data['Industry'] = info.get('industry', 'N/A')
        data['Market Cap'] = info.get('marketCap', 'N/A')
        data['Trailing PE'] = info.get('trailingPE', 'N/A')
        data['Forward PE'] = info.get('forwardPE', 'N/A')
        data['Price to Book'] = info.get('priceToBook', 'N/A')
        data['Profit Margins'] = info.get('profitMargins', 'N/A')
        data['Gross Margins'] = info.get('grossMargins', 'N/A')
        data['Operating Margins'] = info.get('operatingMargins', 'N/A')
        data['Return on Assets'] = info.get('returnOnAssets', 'N/A')
        data['Return on Equity'] = info.get('returnOnEquity', 'N/A')
        data['Beta'] = info.get('beta', 'N/A')
        data['Enterprise to Ebitda'] = info.get('enterpriseToEbitda', 'N/A')
        data['Total Revenue'] = info.get('totalRevenue', 'N/A')
        data['Revenue Growth'] = info.get('revenueGrowth', 'N/A')
        data['Earnings Growth'] = info.get('earningsGrowth', 'N/A')
        data['Total Debt'] = info.get('totalDebt', 'N/A')
        data['Debt to Equity'] = info.get('debtToEquity', 'N/A')
        data['Operating Cashflow'] = info.get('operatingCashflow', 'N/A')
        data['Free Cashflow'] = info.get('freeCashflow', 'N/A')
        data['Dividend Yield'] = info.get('dividendYield', 'N/A')
        data['Payout Ratio'] = info.get('payoutRatio', 'N/A')
        
        # Fetch historical data from 2013-01-01 to the latest date, with monthly intervals
        start_date = '2013-01-01'
        end_date = None  # Let yfinance use the most recent data
        historical_data = stock.history(start=start_date, end=end_date, interval='1mo')
        
        # Keep only the 'Close' price
        historical_close = historical_data['Close'].dropna()
        
        # Add historical close prices to the result
        data['Historical Close Prices'] = historical_close.to_dict()
        
    except Exception as e:
        data['Error'] = f"An error occurred with yfinance: {e}"
        
    return data

if __name__ == '__main__':
    # Replace 'AAPL' with your desired stock symbol
    symbol = 'AAPL'  # For Finnhub
    ticker = 'AAPL'  # For yfinance
    
    # Get data from Finnhub
    finnhub_data = get_finnhub_data(symbol)
    
    # Get data from yfinance
    yfinance_data_result = get_yfinance_data(ticker)
    
    # Combine data into a single output
    output = {
        'Finnhub Data': finnhub_data,
        'Yahoo Finance Data': yfinance_data_result
    }
    
    # Print the output
    print(output)


{'Finnhub Data': {'Company Profile': {'country': 'US', 'currency': 'USD', 'estimateCurrency': 'USD', 'exchange': 'NASDAQ NMS - GLOBAL MARKET', 'finnhubIndustry': 'Technology', 'ipo': '1980-12-12', 'logo': 'https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AAPL.png', 'marketCapitalization': 3587438.2182262465, 'name': 'Apple Inc', 'phone': '14089961010', 'shareOutstanding': 15115.82, 'ticker': 'AAPL', 'weburl': 'https://www.apple.com/'}, 'Financials Reported': {'cik': '320193', 'data': [{'accessNumber': '0000320193-24-000123', 'symbol': 'AAPL', 'cik': '320193', 'year': 2024, 'quarter': 0, 'form': '10-K', 'startDate': '2023-10-01 00:00:00', 'endDate': '2024-09-28 00:00:00', 'filedDate': '2024-11-01 00:00:00', 'acceptedDate': '2024-11-01 06:01:36', 'report': {'bs': [{'concept': 'us-gaap_CashAndCashEquivalentsAtCarryingValue', 'unit': 'usd', 'label': 'Cash and cash equivalents', 'value': 29943000000.0}, {'concept': 'us-gaap_MarketableSecuritiesCurrent', 'unit': 'usd', 'l